In [1]:
import requests

url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
r = requests.get(url)
text = r.text

In [2]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
char2idx = {ch:i for i,ch in enumerate(chars)}
idx2char = {i:ch for i,ch in enumerate(chars)}

encode = lambda x: [char2idx[ch] for ch in x]
decode = lambda x: ''.join([idx2char[idx] for idx in x])

In [3]:
import torch
import numpy as np

data = encode(text)
data = torch.tensor(data, dtype=torch.int64)

In [4]:
split = 0.8  # train/test split
block_size = 8  # context length
batch_size = 4  # number of sequences handled in parallel

In [5]:
train_data = data[:int(len(data)*split)]
val_data = data[int(len(data)*split):]

In [6]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When input {context}, the target is {target}")

When input tensor([18]), the target is 47
When input tensor([18, 47]), the target is 56
When input tensor([18, 47, 56]), the target is 57
When input tensor([18, 47, 56, 57]), the target is 58
When input tensor([18, 47, 56, 57, 58]), the target is 1
When input tensor([18, 47, 56, 57, 58,  1]), the target is 15
When input tensor([18, 47, 56, 57, 58,  1, 15]), the target is 47
When input tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target is 58


In [7]:
torch.manual_seed(1337)

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[58, 63,  8,  0,  0, 19, 24, 27],
        [39, 59, 45, 46, 58,  1, 46, 43],
        [49, 43, 57,  1, 53, 50, 42,  1],
        [52, 41, 47, 43, 52, 58,  1, 56]])
targets:
torch.Size([4, 8])
tensor([[63,  8,  0,  0, 19, 24, 27, 33],
        [59, 45, 46, 58,  1, 46, 43,  1],
        [43, 57,  1, 53, 50, 42,  1, 46],
        [41, 47, 43, 52, 58,  1, 56, 47]])
----
when input is [58] the target: 63
when input is [58, 63] the target: 8
when input is [58, 63, 8] the target: 0
when input is [58, 63, 8, 0] the target: 0
when input is [58, 63, 8, 0, 0] the target: 19
when input is [58, 63, 8, 0, 0, 19] the target: 24
when input is [58, 63, 8, 0, 0, 19, 24] the target: 27
when input is [58, 63, 8, 0, 0, 19, 24, 27] the target: 33
when input is [39] the target: 59
when input is [39, 59] the target: 45
when input is [39, 59, 45] the target: 46
when input is [39, 59, 45, 46] the target: 58
when input is [39, 59, 45, 46, 58] the target: 1
when input is [39, 59, 45, 

In [8]:
print(xb)

tensor([[58, 63,  8,  0,  0, 19, 24, 27],
        [39, 59, 45, 46, 58,  1, 46, 43],
        [49, 43, 57,  1, 53, 50, 42,  1],
        [52, 41, 47, 43, 52, 58,  1, 56]])


In [13]:
import torch.nn as nn
from torch.nn import functional as F

class BigramLM(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()  # functionality of nn.Module
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)  # each token directly reads off the logits for the next token from a lookup table
    
    def forward(self, idx, targets=None):
        # idx and targets are both (B, T) integer tensors
        logits = self.token_embedding_table(idx)  # (B atch * T ime * (C hannels = V ocabulary size))

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is a (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            logits, loss = self(idx)  # get the predictions
            logits = logits[:, -1, :]  # get the logits for the last token
            probs = F.softmax(logits, dim=1)  # convert to probabilities
            idx_next = torch.multinomial(probs, 1)  # sample from the distribution - (B, 1) tensor
            idx = torch.cat([idx, idx_next], dim=1)  # append the new token to the context
        return idx
    
m = BigramLM(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.6693, grad_fn=<NllLossBackward0>)

ZbaKxi3 R?bhdre3;xB&UlM
;'ypiBuVDnukTbxCYdPl!o&bGAp-:ZXXU.Hwt-nyyEt.WBGjGjyqaJw3zg:Oaaf,EbY&nB3bqReb
